In [ ]:
# import and run the stokes1 script
import stokes1
import eztfem as ezt

results = stokes1.main()

max_omega = results[0]
mesh      = results[1]
problem   = results[2]
u         = results[3]
derivs    = results[4]

In [ ]:
# maximum value of omega (vorticity)
print(max_omega)

In [ ]:
# create a PyVista mesh for visualizing results
mesh_pv = ezt.generate_pyvista_mesh(mesh)

In [ ]:
# plot the mesh in Pyvista
ezt.plot_mesh_pv(mesh_pv, show_edges=True, window_size=(800, 400))

In [ ]:
# plot the x-velocity field
ezt.plot_sol(mesh_pv, problem, u, show_scalar_bar=True, n_colors=16,
             window_size=(800, 400), physq=0, degfd=0)

In [ ]:
# plot contours of the x-velocity
ezt.plot_sol_contour(mesh_pv, problem, u, physq=0, degfd=0, nlevels=20)

In [ ]:
# quiver plot of the velocity field
ezt.plot_quiver(mesh_pv, problem, u, window_size=(800, 400), physq=0, 
                scale=0.1)

In [ ]:
# plot the effective strain rate vector
ezt.plot_vector(mesh_pv, problem, derivs["gammadot"])

In [ ]:
# contour plot of the effective strain rate
ezt.plot_vector_contours(mesh_pv, problem, derivs["gammadot"], nlevels=8)

In [ ]:
# sample effective strain rate along a line
points = [[0.0, 0.3, 0.0], [1.0, 0.3, 0.0]]
_ = ezt.plot_vector_over_line(mesh_pv, problem, derivs["gammadot"], points,
                              plot_mesh=False)